In [1]:
# VacationPy
# Note : Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [3]:
# Store Part I results into DataFrame
# Load the csv exported in Part I to a DataFrame

file = pd.read_csv("cities.csv")
file.dropna()
file.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [4]:
# Humidity Heatmap
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
# Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [6]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.

temp_file=file.loc[(file['Max Temp']>=70)&(file['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59


In [8]:
# Hotel Map
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -18.1326021,
                    "lng": 13.882402
                },
                "viewport": {
                    "northeast": {
                        "lat": -18.13097267010728,
                        "lng": 13.88388602989272
                    },
                    "southwest": {
                        "lat": -18.13367232989272,
                        "lng": 13.88118637010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "baa79eb1237e8b4d196fc6c2ea4f75a18ac9e9e2",
            "name": "Hotel Pension Le Manoir",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2448,
                    "html_attributions": [
               

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAEdDSborfdnvPv41z5SOmn8R-CwZ_dOIAc_fHfYC6_FC-3vAeof8FQXeJUl1n8F3d87hGCnvMp0UPlWcgoDop1ECJNiAg3RFW-QvlB7-8PCBvDTyXVEUx4DoP3Q8Hi3jaC-4CW-cUN2jJjTrJvG1g54IWlJWYZKEOB7t8wFBQb8Z9A44spY5hBC7wowjx-55Gsc3FzNlWba7i20_7o63XKUK0pwbN7VKtGj50anM9ibplecPx1c3ylcVPZA0gNZH3twPhEjZ7qLkf8_74bs_71QbF3eYI4CcDihd6OWWau3Lu6jUviPWMfBne1M_xH-aJd9eMM3NCIjPMoPGr0UVqyY8XWzlLPeUKSk7b-z7AhLJDn6hU_Q1UNaQo6hZXyW6EcTPNnoJdZBFYmMMM7jAmH0SEFku-YNCHJqw0k5SwzHCTC0aFARhfO5_7JUY-81SLaNjSuBxOURy",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -20.536313,
                    "lng": -45.606485
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.53466567010728,
                        "lng": -45.60516722010728
                    },
                    "southwest": {
                        "lat": -20.53736532989272,
                    

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAJG3cCDgBB7Gfdr8WFhv8W7NVv73-QtGhQGYjLYdGFCwjFWXOYeBfk5d1RLC5MyHVy6Bjr5QGFYVO4Pucf7euivt17fO4kQbF6Bv09UQxDwGUCMdlWvC0S4SIf_wd-zWzHePvtSCo7INka0zdYl8bEAlLlNpYJ4QE4Jl4grP2KfvO22tPbsYGTB5_3obRDDmpF3M0guyBTYASbZ3458yWb-6HyPMowCEDakDp30-LK15AeXS3erxUq7C6X3_ToDSiIsMIZ-5tuIr5BPSVvBH5UwHVeHnWQy-9nWNsZpQC984Ks-ybSUWDxknsOB7yl93bGwqIBkul_73Y6TuOvQpQA0s15DgNjJLx4asDQ7KyZU1RXdjYiJqJbI5w7sltLwVBzaka5GZCCtxWlLE11nkFgcSENwLptplP6GiaW6C8fiTVFYaFOlPGaQdYm_2rpVSPWZqBE3CIzZf",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -29.9881927,
                    "lng": -50.1398742
                },
                "viewport": {
                    "northeast": {
                        "lat": -29.98678887010728,
                        "lng": -50.13855022010728
                    },
                    "southwest": {
                        "lat": -29.98948852989272,
                  

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAALLLnpeC_sl9-Bt0Jo-riM9saaQOvaqosGqVRxce9D0jsj2xJKbNzOSzp__AMDxJ4qnlOFGcMLW1neYRpTUhLxv-GneC9xAKe0YwxtqEoIebyDMkjtvD4ofjEoAMfjX-Rb9x5n5zgHMJ1mELrmyKKOFSQCVTfGb_ChNmnnsAL7qcmSeO8nT_iG7t-gnhgDIUFB7tI5n6k15-d2RIMKSBfshe02uqLszTYyiJsy29XE3wNdzpijikojRW_iqnnPQZby3VbXbI0ylkXCwoR-Udkjtv1d1y3-OfxkwxFpT0_wpiIV7yXPw6HYZ0I32_fiwfnGsvqPVX3cfCW850zd9EL4uF3XwztGHKix9If_e4eDUmDj546x0CAW6kJLTadGmGJfD81XK1mxbxvcDYqpm9QugSEGBR1O8om__-Be6kdR3IODUaFEPZRs8RQSXPsSGyoqttl9MZVZgj",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -20.290218,
                    "lng": 44.299996
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.28947867010728,
                        "lng": 44.30136652989273
                    },
                    "southwest": {
                        "lat": -20.29217832989272,
                      

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 15.0040551,
                    "lng": 74.032279
                },
                "viewport": {
                    "northeast": {
                        "lat": 15.00535382989272,
                        "lng": 74.03354577989272
                    },
                    "southwest": {
                        "lat": 15.00265417010728,
                        "lng": 74.03084612010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "7481f733341b77b9956a3badfe6ac2b68e6e8122",
            "name": "Oceanic Hotel",
            "photos": [
                {
                    "height": 3024,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/113116447481431456013/photos\">Ocea

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAK-Jo3XQAOK8BikYgeMW6XZju9BLYPe-TNeS2HBFfKuhaFJ9tVlf5nxCp2kh_gFySMcNaF4Wdj2gRSr4S5ZfPmZ3W2TcrA-DkgBzaEhD6qErcP66VotNutg80wOLabLBLQvndOu4pXrO4zkbjzxWbZZI7e0bvN3H2veBNI97iIeFrT28RNZvF_epFeqECK-8r2q-UX7PUpoJcIlXxQWtOe16nFrDDkJrxuqSM9BSJK5onVrxyHdxRdlHMdckXa2PZTCi1tYjNgmN7_M094ihySGqB9hZMtHUfkzi3itZBSFpvQYufaxt4FUaUDUQ6gsHn4Plwav4d2H91rhH-46uWhQashQYV2-YGx7h4Un6zh2t4Skg5X66Dn-Z9Q9eh3kvYBU4twoVJozFG1oPj-VS_9MSECFv6mgAqTaO0YXo8biuV_gaFK1qpMGHM_3Q6IBdqW3b3IWGxg_Q",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 35.296321,
                    "lng": -2.937649
                },
                "viewport": {
                    "northeast": {
                        "lat": 35.29770157989272,
                        "lng": -2.936256470107277
                    },
                    "southwest": {
                        "lat": 35.29500192010728,
                        

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 22.9164309,
                    "lng": 96.4563258
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.9177379,
                        "lng": 96.45768732989272
                    },
                    "southwest": {
                        "lat": 22.9125099,
                        "lng": 96.45498767010729
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "f63499003fa624669e9c027aebacd2f2a4a8d167",
            "name": "Golden Butterfly Hotel",
            "photos": [
                {
                    "height": 1108,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/112290800190781055439/photos\">\u53ef\u

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 32.8802013,
                    "lng": 59.2162932
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.88151887989271,
                        "lng": 59.21771767989272
                    },
                    "southwest": {
                        "lat": 32.87881922010727,
                        "lng": 59.21501802010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "cd69218f80a5fa88a47d8ceaf8d490cee2d4283f",
            "name": "\u0647\u062a\u0644 \u067e\u0631\u0648\u064a\u0632",
            "place_id": "ChIJ8dNk011nGj8RZI51wiSCBsY",
            "plus_code": {
                "compound_code": "V6J8+3G Birjand, South Khorasan Province, Iran",
                "global_co

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,Hotel Pension Le Manoir
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Furnaspark Resort
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Kimar
3,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Hotel maeva morondava
4,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Oceanic Hotel
5,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Parador de Melilla
6,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Golden Butterfly Hotel
7,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,هتل پرويز


In [9]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))